<a href="https://colab.research.google.com/github/EgorGod21/Comparison_detection_models/blob/main/Models/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torchvision.transforms as T
import torchvision
import numpy as np
import cv2
import os
import time
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-12-23 Python-3.10.12 torch-2.1.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 68.4MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
# Images
imgs = ['https://ultralytics.com/images/zidane.jpg', 'https://ultralytics.com/images/bus.jpg']  # batch of images

# Inference
results = model(imgs)


print(results.xyxy[0])
results.pandas().xyxy[0]

tensor([[7.42897e+02, 4.79784e+01, 1.14114e+03, 7.16823e+02, 8.80724e-01, 0.00000e+00],
        [4.42039e+02, 4.37350e+02, 4.96720e+02, 7.09883e+02, 6.87266e-01, 2.70000e+01],
        [1.25215e+02, 1.93607e+02, 7.10841e+02, 7.13070e+02, 6.42235e-01, 0.00000e+00],
        [9.82893e+02, 3.08405e+02, 1.02733e+03, 4.20230e+02, 2.63047e-01, 2.70000e+01]])


,xmin,ymin,xmax,ymax,confidence,class,name
0,742.896973,47.978394,1141.142212,716.822998,0.880724,0,person
1,442.038971,437.349854,496.719574,709.882935,0.687266,27,tie
2,125.215088,193.606750,710.841187,713.070251,0.642235,0,person
3,982.893311,308.404541,1027.329590,420.230103,0.263047,27,tie


In [4]:
coco_classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
    "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]


In [11]:
results = []

def detect_object(img_path, output_path, confidence=0.5, rect_th=2, text_size=1, text_th=2):

    model.cpu()

    pred = model(img_path)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    pred_cls = []
    for i in pred.xyxy[0]:
      if i[4] > confidence:
        pt1 = tuple(map(int, (i[0], i[1])))
        pt2 = tuple(map(int, (i[2], i[3])))
        cv2.rectangle(img, pt1, pt2, color=(0, 255, 0), thickness=rect_th)
        cv2.putText(img, coco_classes[int(i[5])], pt1, cv2.FONT_HERSHEY_SIMPLEX, text_size, (0, 255, 0), thickness=text_th)
        pred_cls.append(coco_classes[int(i[5])])

    file_name = os.path.basename(img_path)

    result = {
            'Model': 'YOLO',
            'Image': file_name,
            'Predicted Classes': dict(Counter(pred_cls)),
            'Number of Objects': len(pred_cls)
        }
    results.append(result)
    print(result)

    output_path = os.path.join(output_path, file_name)

    plt.figure(figsize=(20, 30))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0.0, transparent=True)
    plt.show()

In [7]:
output_yolo = './TestPhotoPredictYOLO'
os.makedirs(output_yolo, exist_ok=True)

In [12]:
for i in range(1,33):
  detect_object(f'./TestPhoto/{i}.jpg', output_yolo, confidence=0.6)

{'Model': 'YOLO', 'Image': '1.jpg', 'Predicted Classes': {'car': 5}, 'Number of Objects': 5}
{'Model': 'YOLO', 'Image': '2.jpg', 'Predicted Classes': {'car': 2, 'truck': 1}, 'Number of Objects': 3}
{'Model': 'YOLO', 'Image': '3.jpg', 'Predicted Classes': {'car': 2}, 'Number of Objects': 2}
{'Model': 'YOLO', 'Image': '4.jpg', 'Predicted Classes': {'car': 3}, 'Number of Objects': 3}
{'Model': 'YOLO', 'Image': '5.jpg', 'Predicted Classes': {'car': 5, 'truck': 1}, 'Number of Objects': 6}
{'Model': 'YOLO', 'Image': '6.jpg', 'Predicted Classes': {'car': 6}, 'Number of Objects': 6}
{'Model': 'YOLO', 'Image': '7.jpg', 'Predicted Classes': {'car': 1}, 'Number of Objects': 1}
{'Model': 'YOLO', 'Image': '8.jpg', 'Predicted Classes': {'stop sign': 1}, 'Number of Objects': 1}
{'Model': 'YOLO', 'Image': '9.jpg', 'Predicted Classes': {'truck': 1, 'person': 5, 'traffic light': 1, 'car': 1}, 'Number of Objects': 8}
{'Model': 'YOLO', 'Image': '10.jpg', 'Predicted Classes': {'traffic light': 3, 'car': 1}

In [ ]:
df = pd.DataFrame(results)
df

,Model,Image,Predicted Classes,Number of Objects
0,YOLO,1.jpg,{'car': 5},5
1,YOLO,2.jpg,"{'car': 2, 'truck': 1}",3
2,YOLO,3.jpg,{'car': 2},2
3,YOLO,4.jpg,{'car': 3},3
4,YOLO,5.jpg,"{'car': 5, 'truck': 1}",6
5,YOLO,6.jpg,{'car': 6},6
6,YOLO,7.jpg,{'car': 1},1
7,YOLO,8.jpg,{'stop sign': 1},1
8,YOLO,9.jpg,"{'truck': 1, 'person': 5, 'traffic light': 1, ...",8
9,YOLO,10.jpg,"{'traffic light': 3, 'car': 1}",4


In [ ]:
df.to_csv(f'{output_yolo}/models_info.csv', index=False)

In [ ]:
def check_inference_time(image_path, gpu=False):
  if gpu:
    model.cuda()
  else:
    model.cpu()
  start_time = time.time()
  pred = model(image_path)
  end_time = time.time()
  return end_time-start_time

In [ ]:
gpu_time = sum([check_inference_time(f'./TestPhoto/{i}.jpg', gpu=True) for i in range(1, 11)])/10.0
cpu_time = sum([check_inference_time(f'./TestPhoto/{i}.jpg', gpu=False) for i in range(1, 11)])/10.0

print('\n\nAverage Time take by the model with GPU = {}s\nAverage Time take by the model with CPU = {}s'.format(gpu_time,cpu_time))



Average Time take by the model with GPU = 0.019626617431640625s
Average Time take by the model with CPU = 0.21328997611999512s


In [ ]:
data = {'Model': ['YOLO'], 'Inference_Time_GPU': [gpu_time], 'Inference_Time_CPU': [cpu_time]}
data

{'Model': ['YOLO'],
 'Inference_Time_GPU': [0.019626617431640625],
 'Inference_Time_CPU': [0.21328997611999512]}

In [ ]:
df_time = pd.DataFrame(data)
df_time.to_csv(f'{output_yolo}/models_time.csv', index=False)